In [390]:
import torch
import numpy as np

# Tensori

In [391]:
t1 = torch.tensor(4.)
t1

tensor(4.)

In [392]:
t2 = torch.tensor([1., 2, 3, 4])
#NB! Tutti i dati di un tensore hanno sempre lo stesso tipo, in questo caso anche gli int diventano dei float

In [393]:
t3 =torch.tensor([[5., 6],
                  [7, 8],
                  [9, 10]
                ])
t3

tensor([[ 5.,  6.],
        [ 7.,  8.],
        [ 9., 10.]])

In [394]:
t4 = torch.tensor([
    [[11, 12, 13],
     [13, 14, 15]],
    [[15, 16, 17],
     [17, 18, 19]]
])
t4

#crea una matrice tridimensionale 2x2x3 (2 "strati", 2 righe [righe in ogni strato], 3 colonne [elementi all'interno di ogni riga])

tensor([[[11, 12, 13],
         [13, 14, 15]],

        [[15, 16, 17],
         [17, 18, 19]]])

In [395]:
print(t1)
t1.shape

tensor(4.)


torch.Size([])

In [396]:
print(t2)
t2.shape

tensor([1., 2., 3., 4.])


torch.Size([4])

In [397]:
print(t3)
t3.shape

tensor([[ 5.,  6.],
        [ 7.,  8.],
        [ 9., 10.]])


torch.Size([3, 2])

In [398]:
print(t4)
t4.shape

tensor([[[11, 12, 13],
         [13, 14, 15]],

        [[15, 16, 17],
         [17, 18, 19]]])


torch.Size([2, 2, 3])

In [399]:
# NB! NON si possono creare tensori con shape non definiti, ossia non si possono lasciare "spazi vuoti", se la prima riga
# è [1, 2, 3] la seconda non può essere ad esempio [5, 6] perché mancherebbe un elemento per completare la riga

In [400]:
xt = torch.tensor (3. )
wt = torch. tensor (4., requires_grad =True)
bt = torch.tensor (5., requires_grad = True)
xt, wt, bt

# requires_grad =True specifica che siamo interessati a calcolare le derivate di questa variabile all'interno del programma
# se non lo mettiamo il programma non calcolerà mai la derivata rispetto quella determinata variabile

(tensor(3.), tensor(4., requires_grad=True), tensor(5., requires_grad=True))

In [401]:
yt = wt * xt + bt
yt

tensor(17., grad_fn=<AddBackward0>)

In [402]:
yt.backward()

#esegue la derivata del tensore e salva i risultati all'interno della proprietà .grad del tensore

In [403]:
print('dy/dx: ', xt.grad)
print('dy/dw: ', wt.grad)
print('dy/db: ', bt.grad)

dy/dx:  None
dy/dw:  tensor(3.)
dy/db:  tensor(1.)


In [404]:
t6 = torch.full((3,2), 11)
t6

# torch.full((shape), num che inserisce)        serve a riempire un tensore con un numero specificando una shape

tensor([[11, 11],
        [11, 11],
        [11, 11]])

In [405]:
t7 = torch.cat((t3, t6))
t7

# torch.cat((tensore1, tensore2))       permette di concatenare due tensori NB! devono avere shape compatibili

tensor([[ 5.,  6.],
        [ 7.,  8.],
        [ 9., 10.],
        [11., 11.],
        [11., 11.],
        [11., 11.]])

In [406]:
t8 = torch.sin(t7)
t8

# torch.sin(tensore)     calcola il seno di ogni valore nel tensore

tensor([[-0.9589, -0.2794],
        [ 0.6570,  0.9894],
        [ 0.4121, -0.5440],
        [-1.0000, -1.0000],
        [-1.0000, -1.0000],
        [-1.0000, -1.0000]])

In [407]:
t9 = t8.reshape(3, 2, 2)
t9

# tensore.reshape(3, 2, 2)      permette di assegnare una nuova shape a un tensore NB! devono essere compatibili il numero di dati

tensor([[[-0.9589, -0.2794],
         [ 0.6570,  0.9894]],

        [[ 0.4121, -0.5440],
         [-1.0000, -1.0000]],

        [[-1.0000, -1.0000],
         [-1.0000, -1.0000]]])

# Numpy

In [408]:
a = np.array([[1,2],
              [3, 4]])
a

array([[1, 2],
       [3, 4]])

In [409]:
b = torch.from_numpy(a)
b

# torch.from_numpy(var_numpy)       trasforma un vettore numpy in un tensore torch

tensor([[1, 2],
        [3, 4]])

In [410]:
c = b.numpy()
c

# tensore.numpy()       trasforma un tensore in un array numpy

array([[1, 2],
       [3, 4]])

# Regressione lineare

In una regressione lineare ogni target è stimato da una somma pesata delle variabili in input a cui è sommata una distorsione detta bias
Il training della regressione lineare avviene tramite il settaggio dei pesi e dei bias affinchè la predizione di nuove informazioni sia accurata

In [411]:
# Input (temp, precipitazioni, umidità)

inputs = np.array([[72, 67, 43],
                   [91, 88, 64],
                   [87, 134, 58],
                   [102, 43, 37],
                   [69, 96, 70]],
                   dtype = 'float32'
                  )

In [412]:
# obiettivi (mele, arance)

obiettivi = np.array([[56, 70],
                    [81, 101],
                    [119, 133],
                    [22, 37],
                    [103, 119]],
                    dtype = 'float32'
                   )

Solitamente i dati vengono importati da file csv in array numpy che vengono processati e infine convertiti in PyTorch

In [413]:
inputs = torch.from_numpy(inputs)
obiettivi = torch.from_numpy(obiettivi)

print(inputs)
print(obiettivi)

tensor([[ 72.,  67.,  43.],
        [ 91.,  88.,  64.],
        [ 87., 134.,  58.],
        [102.,  43.,  37.],
        [ 69.,  96.,  70.]])
tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.]])


In [414]:
pesi = torch.randn(2, 3, requires_grad = True)
bias = torch.randn(2, requires_grad = True)

# torch.randn(shape, requires_grad = True)      crea un tensore con la shape data con elementi random che hanno una distribuzione normalizzata

print(pesi)
print(bias)

tensor([[-0.8632, -0.1435, -0.9080],
        [-0.6156,  0.1464,  0.0685]], requires_grad=True)
tensor([1.1616, 2.0586], requires_grad=True)


Funzione che fa una predizione delle variabili target in base agli input dati

In [415]:
def model(x):
    return x @ pesi.t() + bias

# @ moltiplicazione matriciale in PyTorch
# .t metodo che fa la trasposta del tensore

In [416]:
pred = model(inputs)
print(pred)

tensor([[-109.6432,  -29.5107],
        [-148.1248,  -36.6938],
        [-145.8233,  -27.9082],
        [-126.6463,  -51.9047],
        [-135.7314,  -21.5674]], grad_fn=<AddBackward0>)


Per migliorare la predizione dobbiamo prima compararla al target che ci si aspettava attraverso una funzione di costo come l'errore medio quadratico

In [417]:
def mse(t1, t2):
    diff = t1 - t2
    return torch.sum(diff * diff) / diff.numel()

#torch.sum      restituisce la somma di tutti gli elementi del tensore
# tensore.numel     restituisce numero di elementi all'interno del tensore

In [418]:
perdita = mse(pred, obiettivi)
print(perdita)

tensor(31157.2285, grad_fn=<DivBackward0>)


Si ha una predizione migliore quanto più è minore la perdita calcolata

In [419]:
perdita.backward()

In [420]:
print(pesi)
print(pesi.grad)

tensor([[-0.8632, -0.1435, -0.9080],
        [-0.6156,  0.1464,  0.0685]], requires_grad=True)
tensor([[-17490.1387, -19211.4824, -11871.5020],
        [-10492.2695, -11532.6680,  -7110.6460]])


In [421]:
print(bias)
print(bias.grad)

tensor([1.1616, 2.0586], requires_grad=True)
tensor([-209.3938, -125.5170])


In [422]:
with torch.no_grad():
    pesi -= pesi.grad * 1e-5
    bias -= bias.grad * 1e-5

# with torch.no_grad():         serve per non modificare la derivata mentre si aggiornano pesi e bias

In [423]:
pred = model(inputs)
perdita = mse(pred, obiettivi)
print(perdita)

# la perdita diminuisce rispetto alla perdita precedente

tensor(21053.0039, grad_fn=<DivBackward0>)


In [424]:
pesi.grad.zero_()
bias.grad.zero_()

print(pesi.grad)
print(bias.grad)

# bisogna resettare le derivate a zero perché pytorch accumula le derivate (derivata prima, seconda...)

tensor([[0., 0., 0.],
        [0., 0., 0.]])
tensor([0., 0.])


## Training

1. gerenare delle predizioni
2. calcolare la perdita
3. calcolare le derivata dei pesi e dei bias
4. calibrare pesi e bias sottraendo una piccola porzione della derivata
5. resettare la derivata a zero

In [425]:
# Generazione predizioni

pred = model(inputs)
print(pred)

tensor([[ -79.0718,  -11.1705],
        [-107.7028,  -12.4451],
        [ -97.9760,    0.7993],
        [ -96.1509,  -33.6113],
        [ -96.9081,    1.7224]], grad_fn=<AddBackward0>)


In [426]:
# Calcolo della perdita

perdita = mse(pred, obiettivi)
print(perdita)

tensor(21053.0039, grad_fn=<DivBackward0>)


In [427]:
print(pesi)
print(bias)

tensor([[-0.6882,  0.0486, -0.7893],
        [-0.5107,  0.2617,  0.1397]], requires_grad=True)
tensor([1.1637, 2.0599], requires_grad=True)


In [428]:
# Calcolo derivata

perdita.backward()
print(pesi.grad)
print(bias.grad)

tensor([[-14323.8164, -15800.4209,  -9766.9648],
        [ -8592.7510,  -9486.2852,  -5848.1025]])
tensor([-171.7619, -102.9411])


In [429]:
# Calibrazione pesi e bias

with torch.no_grad():
    pesi -= pesi.grad * 1e-5        #perché mi da errore?: variable is 'unbound' ma il programma funziona lo stesso
    bias -= bias.grad * 1e-5

In [430]:
# Azzeramento pesi e bias

pesi.grad.zero_()
bias.grad.zero_()

tensor([0., 0.])

In [431]:
# visualizzazione nuovi pesi e bias calibrati

print(pesi)
print(bias)

tensor([[-0.5450,  0.2067, -0.6917],
        [-0.4248,  0.3566,  0.1981]], requires_grad=True)
tensor([1.1654, 2.0609], requires_grad=True)


In [432]:
pred = model(inputs)
perdita = mse(pred, obiettivi)
print(perdita)

tensor(14239.1426, grad_fn=<DivBackward0>)


## Training per più epochs

In [433]:
for i in range(100):
    pred = model(inputs)
    perdita = mse(pred, obiettivi)
    perdita.backward()

    with torch.no_grad():
        pesi -= pesi.grad * 1e-5
        bias -= bias.grad * 1e-5
        pesi.grad.zero_()
        bias.grad.zero_()

In [434]:
pred = model(inputs)
perdita = mse(pred, obiettivi)
print(perdita)

tensor(68.2037, grad_fn=<DivBackward0>)


In [435]:
pred

tensor([[ 59.8985,  72.1854],
        [ 77.3146,  98.4472],
        [125.4897, 135.1190],
        [ 34.8994,  44.9135],
        [ 85.5778, 111.0749]], grad_fn=<AddBackward0>)

In [436]:
obiettivi

tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.]])

# Regressione lineare usando funzioni già presenti su PyTorch

In [437]:
import torch.nn as nn

In [438]:
# inizializzazione input e obiettivi


inputs = np.array([[73, 67, 43],
                   [91, 88, 64],
                   [87, 134, 58],
                   [102, 43, 37],
                   [69, 96, 70],
                   [74, 66, 43],
                   [91, 86, 65],
                   [88, 134, 59],
                   [101, 44, 37],
                   [68, 96, 71],
                   [73, 66, 44],
                   [92, 87, 64],
                   [87, 135, 57],
                   [103, 43, 36],
                   [68, 97, 70]],
                   dtype = 'float32'
                  )

obiettivi = np.array([[56, 70],
                    [81, 101],
                    [119, 133],
                    [22, 37],
                    [103, 119],
                    [57, 69],
                    [80, 102],
                    [118, 132],
                    [21, 38],
                    [104, 118],
                    [57, 69],
                    [82, 100],
                    [118, 134],
                    [20, 38],
                    [102, 120]],
                    dtype = 'float32'
                   )

In [439]:
# Trasofrmazione degli input e obiettivi a tensori
inputs = torch.from_numpy(inputs)
obiettivi = torch.from_numpy(obiettivi)


## Dataset, DataLoader e regressione lineare

In [440]:
from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader

In [441]:
# definizione del dataset
train_ds = TensorDataset(inputs, obiettivi)
train_ds[0:5]

#TensorDataSet permette di accedere a piccole porzioni del data training usando la notazione array

(tensor([[ 73.,  67.,  43.],
         [ 91.,  88.,  64.],
         [ 87., 134.,  58.],
         [102.,  43.,  37.],
         [ 69.,  96.,  70.]]),
 tensor([[ 56.,  70.],
         [ 81., 101.],
         [119., 133.],
         [ 22.,  37.],
         [103., 119.]]))

In [442]:
# definizione data loader

batch_size = 5
train_dl = DataLoader(train_ds, batch_size, shuffle = True)

# DataLoader spezza il set di dati in lotti di una dimensione predefinita durante il training

In [443]:
for xb, yb in train_dl:
    print(xb)
    print(yb)
    break

tensor([[103.,  43.,  36.],
        [ 92.,  87.,  64.],
        [102.,  43.,  37.],
        [ 73.,  67.,  43.],
        [ 91.,  86.,  65.]])
tensor([[ 20.,  38.],
        [ 82., 100.],
        [ 22.,  37.],
        [ 56.,  70.],
        [ 80., 102.]])


In [444]:
# definizione del modello

model = nn.Linear(3, 2)     #nn.Linear(numero input, numero output)
print(model.weight)
print(model.bias)

print(model.weight.shape)

Parameter containing:
tensor([[ 0.3970, -0.5649,  0.3907],
        [ 0.1361,  0.5138,  0.2441]], requires_grad=True)
Parameter containing:
tensor([-0.4026, -0.0020], requires_grad=True)
torch.Size([2, 3])


In [ ]:
list()

In [445]:
model.weight.shape

torch.Size([2, 3])

In [446]:
model.bias.shape

torch.Size([2])

In [447]:
pred = model(inputs)
print(pred)

tensor([[  7.5356,  54.8581],
        [ 11.0251,  73.2251],
        [-18.8910,  94.8521],
        [ 30.2616,  45.0086],
        [  0.1160,  75.8061],
        [  8.4974,  54.4804],
        [ 12.5455,  72.4416],
        [-18.1033,  95.2323],
        [ 29.2997,  45.3864],
        [  0.1097,  75.9141],
        [  8.4911,  54.5884],
        [ 11.9870,  72.8474],
        [-19.8466,  95.1218],
        [ 30.2679,  44.9006],
        [ -0.8459,  76.1838]], grad_fn=<AddmmBackward0>)


## Funzione di perdita

In [448]:
import torch.nn.functional as F

In [449]:
funz_perdita = F.mse_loss

# definizione funzione di perdita con mse

In [450]:
perdita = funz_perdita(model(inputs), obiettivi)
print(perdita)

tensor(4107.0879, grad_fn=<MseLossBackward0>)


## Ottimizzazione perdita

In [451]:
ott = torch.optim.SGD(model.parameters(), lr = 1e-5)

# Funzione per training

In [452]:
def fit(num_epochs, model, funz_perdita, ott, train_dl):
    for epoch in range(num_epochs):
        for xb, yb in train_dl:
            
            # 1. genreazione predizioni
            pred = model(xb)
            
            # 2. calcolo perdita
            perdita = funz_perdita(pred, yb)
            
            # 3. calcolo derivata
            perdita.backward()
            
            # 4. aggiornamento parametri usando derivate
            ott.step()
            
            # reset derivate a zero
            ott.zero_grad()
        if(epoch+1) % 10 == 0:
            print('Epoch [{}/{}], Perdita: {:.4f}'.format(epoch+1, num_epochs, perdita.item()))

In [453]:
fit(100, model, funz_perdita, ott, train_dl)

Epoch [10/100], Perdita: 732.0026
Epoch [20/100], Perdita: 160.7530
Epoch [30/100], Perdita: 462.4055
Epoch [40/100], Perdita: 197.0450
Epoch [50/100], Perdita: 136.1533
Epoch [60/100], Perdita: 156.8247
Epoch [70/100], Perdita: 56.5696
Epoch [80/100], Perdita: 75.0165
Epoch [90/100], Perdita: 44.7002
Epoch [100/100], Perdita: 26.6781


In [454]:
pred = model(inputs)
pred

tensor([[ 58.7101,  71.4303],
        [ 83.8615,  98.2213],
        [111.9501, 136.9023],
        [ 30.9102,  43.0715],
        [ 99.0190, 111.3300],
        [ 57.7831,  70.4012],
        [ 83.2166,  96.9307],
        [112.4293, 137.2647],
        [ 31.8372,  44.1006],
        [ 99.9848, 111.9625],
        [ 58.7490,  71.0337],
        [ 82.9345,  97.1922],
        [111.9113, 137.2988],
        [ 29.9444,  42.4389],
        [ 99.9460, 112.3591]], grad_fn=<AddmmBackward0>)

In [455]:
obiettivi

tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.],
        [ 57.,  69.],
        [ 80., 102.],
        [118., 132.],
        [ 21.,  38.],
        [104., 118.],
        [ 57.,  69.],
        [ 82., 100.],
        [118., 134.],
        [ 20.,  38.],
        [102., 120.]])